https://github.com/tomverbeure/math spinalhdl math elements. Paper list


https://archfx.github.io/posts/2023/04/racketutes4/
https://people.csail.mit.edu/nickolai/papers/moroze-rtlv.pdf was this the paper I saw in pldi 2022?

https://github.com/anishathalye/knox
https://github.com/anishathalye/rtlv?tab=readme-ov-file
https://github.com/anishathalye/chroniton

https://github.com/lip6/coriolis  Coriolis VLSI EDA Tool (LIP6)
ghdl - VHDL simulator
vhdl
system verilog

https://github.com/lip6/libDDD data decisionb diagrams and set decision diagrams

# sel4
https://hol-theorem-prover.org/#get
`sudo apt install polyml`
git clone https://github.com/HOL-Theorem-Prover/HOL.git

https://github.com/seL4/graph-refine

seL4 WCET papers mentioned in the seL4 whitepaper:
https://trustworthy.systems/publications/full_text/Sewell_KH_17.pdf
https://trustworthy.systems/publications/nicta_full_text/4863.pdf
https://trustworthy.systems/publications/nicta_full_text/9118.pdf
Also, bibliography here: https://trustworthy.systems/projects/OLD/wcet

https://github.com/HOL-Theorem-Prover/HOL/tree/develop/examples/l3-machine-code/riscv

leino unstructured code 2005 wp
https://dl.acm.org/doi/10.1145/1108768.1108813

use nix?

## real time os
https://en.wikipedia.org/wiki/Real-time_operating_system 
https://en.wikipedia.org/wiki/Comparison_of_real-time_operating_systems
Freertos
RTEMS https://www.rtems.org/
Zehpyr https://en.wikipedia.org/wiki/Zephyr_(operating_system)
vxworks
nuttx
threadx

## FreeRTOS


# CPUs
TD4 verilog
nand2tetris https://github.com/philzook58/nand2coq/tree/master/verilog
take simple riscv cpu

Can I validate a C / Rust model with respect to these?
https://github.com/lowRISC/ibex
https://github.com/YosysHQ/picorv32
https://github.com/YosysHQ/nerv
https://github.com/YosysHQ/riscv-formal/tree/main

## TD4
https://chatgpt.com/c/6720d909-2f44-8008-9ded-17ed900d6719
https://www.philipzucker.com/td4-4bit-cpu/


In [73]:
%%file /tmp/td4.v
module TD4 (
    input clk,
    input rst,
    input [7:0] dip_switch [16],
    input [3:0] in_dip_switch,
    output reg [3:0] out_led,
    output reg [3:0] A,       // Added output for register A
    output reg [3:0] B,       // Added output for register B
    output reg [3:0] PC,      // Added output for program counter
    output reg carry          // Added output for carry flag
);

    // Instruction opcodes
    parameter ADD_A_IM = 4'b0000,  MOV_A_B = 4'b0001,   IN_A = 4'b0010, MOV_A_IM = 4'b0011;
    parameter MOV_B_A =  4'b0100, ADD_B_IM = 4'b0101,   IN_B = 4'b0110, MOV_B_IM = 4'b0111;
    parameter OUT_B =    4'b1001,   OUT_IM = 4'b1011, JNC_IM = 4'b1110,   JMP_IM = 4'b1111;

    // Main CPU cycle
    always @(posedge clk or posedge rst) begin
        if (rst) {A, B, PC, carry, out_led} <= 0;
        else begin
            // Data
            case (dip_switch[PC][7:4])
                ADD_A_IM:  {carry, A} <= A + dip_switch[PC][3:0];
                MOV_A_B:   A <= B;
                IN_A, MOV_A_IM:  A <= dip_switch;
                MOV_B_A:   B <= A;
                ADD_B_IM:  {carry, B} <= B + dip_switch[PC][3:0];
                IN_B, MOV_B_IM:  B <= dip_switch[PC][3:0];
                OUT_B:     out_led <= B;
                OUT_IM:    out_led <= dip_switch;
            endcase
            // Control
            case (dip_switch[PC][7:4])
                JNC_IM:    PC <= !carry ? dip_switch : PC + 1;
                JMP_IM:    PC <= dip_switch;
                default:   PC <= PC + 1;
            endcase
        end
    end
endmodule


Overwriting /tmp/td4.v


In [74]:
%%file /tmp/tb.v
module td4_tb;
    reg clk, rst;
    reg [3:0] dip_switch;
    wire [3:0] out_led;
    wire [3:0] A, B, PC;
    wire carry;

    // Instantiate the TD4 CPU with new outputs for A, B, PC, and carry
    TD4 td4 (
        .clk(clk), .rst(rst), .dip_switch(dip_switch),
        .out_led(out_led), .A(A), .B(B), .PC(PC), .carry(carry)
    );

    // Clock generation
    always #5 clk = ~clk;  // 10ns period (100MHz)

    initial begin
        // Initialize signals
        clk = 0;
        rst = 1;
        dip_switch = 4'b0000;

        // Release reset after some delay
        #20 rst = 0;

        // Test a simple sequence of instructions
        #10 dip_switch = 4'b0000;  // ADD A, Im
        #10 dip_switch = 4'b0001;  // MOV A, B
        #10 dip_switch = 4'b1001;  // OUT B
        #10 dip_switch = 4'b1111;  // JMP 0
        #10 dip_switch = 4'b0000;  // Set dip_switch to zero for testing

        // Run simulation for some cycles
        #1000 $finish;
    end

    // Monitor outputs
    initial begin
        $monitor("Time=%0t | clk=%b | rst=%b | dip_switch=%b | out_led=%b | A=%b | B=%b | PC=%b | carry=%b", 
                 $time, clk, rst, dip_switch, out_led, A, B, PC, carry);
    end
endmodule


Overwriting /tmp/tb.v


In [75]:
! iverilog -o /tmp/tb /tmp/td4.v /tmp/tb.v && vvp /tmp/tb

Time=0 | clk=0 | rst=1 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0000 | carry=0
Time=5 | clk=1 | rst=1 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0000 | carry=0
Time=10 | clk=0 | rst=1 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0000 | carry=0
Time=15 | clk=1 | rst=1 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0000 | carry=0
Time=20 | clk=0 | rst=0 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0000 | carry=0
Time=25 | clk=1 | rst=0 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0001 | carry=0
Time=30 | clk=0 | rst=0 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0001 | carry=0
Time=35 | clk=1 | rst=0 | dip_switch=0000 | out_led=0000 | A=0000 | B=0000 | PC=0010 | carry=0
Time=40 | clk=0 | rst=0 | dip_switch=0001 | out_led=0000 | A=0000 | B=0000 | PC=0010 | carry=0
Time=45 | clk=1 | rst=0 | dip_switch=0001 | out_led=0000 | A=0000 | B=0000 | PC=0011 | carry=0
Time=50 | clk=0 | rst=0 | dip_switch=1001 | out_led=

# EBMC / HWBMC
https://github.com/diffblue/hw-cbmc


In [1]:
%%file /tmp/main.sv
module main(input clk, x, y);

  reg [1:0] cnt1;
  reg z;

  initial cnt1=0;
  initial z=0;

  always @(posedge clk) cnt1=cnt1+1;

  always @(posedge clk)
    casex (cnt1)
      2'b00:;
      2'b01:;
      2'b1?: z=1;
    endcase

  p1: assert property (@(posedge clk) (z==0));

endmodule


Writing /tmp/main.sv


In [5]:
! ebmc --module main /tmp/main.sv 

Parsing /tmp/main.sv
Converting
Type-checking Verilog::main
using default bound 1
Generating Decision Problem
Using MiniSAT 2.2.1 with simplifier
Properties
Solving with propositional reduction
Checking main.p1
SAT checker: instance is UNSATISFIABLE
UNSAT: No counterexample found within bound

** Results:
[main.p1] always main.z == 0: PROVED up to bound 1


In [21]:
! ebmc --module main /tmp/main.sv --bound 3 --waveform

Parsing /tmp/main.sv
Converting
Type-checking Verilog::main
Generating Decision Problem
Using MiniSAT 2.2.1 with simplifier
Properties
Solving with propositional reduction
Checking main.p1
SAT checker: instance is SATISFIABLE
SAT: counterexample found

** Results:
[main.p1] always main.z == 0: REFUTED
Counterexample:
              0     1     2     3
 main.clk                        
main.cnt1 2'b00 2'b01 2'b10 2'b11
   main.x                        
   main.y                        
   main.z     0     0     0     1


In [7]:
! ebmc --help


* *      EBMC - Copyright (C) 2001-2017 Daniel Kroening     * *
* *                     Version 5.2                         * *
* *     University of Oxford, Computer Science Department   * *
* *                  kroening@kroening.com                  * *

Usage:                       Purpose:

 ebmc [-?] [-h] [--help]      show help
 ebmc file ...                source file names

Additonal options:
 --bound nr                   set bound (default: 1)
 --module module              set top module (deprecated)
 --top module                 set top module
 -p expr                      specify a property
 --outfile file name          set output file name (default: stdout)
 --json-result file name      use JSON for property status and traces
 --trace                      generate a trace for failing properties
 --vcd file name              generate traces in VCD format
 --waveform                   show a waveform for failing properties
 --numbered-trace             give a trace with iden

In [20]:
! ebmc --module main /tmp/main.sv --show-trans

Parsing /tmp/main.sv
Converting
Type-checking Verilog::main
Initial state constraints:

  main.z == 0

  main.cnt1 == 2'b00

State constraints:

  main.cnt1_aux0 == main.cnt1 + 2'b01

Transition constraints:

  next(main.z) == (main.cnt1 == 2'b00 ? main.z : main.cnt1 == 2'b01 ? main.z : (main.cnt1 & ~constant
  * type: bv
      * width: 4
      * #verilog_aval_bval: verilog_unsignedbv
  * value: 7[4:2]) == (constant
  * type: bv
      * width: 4
      * #verilog_aval_bval: verilog_unsignedbv
  * value: 7[2:0] & ~constant
  * type: bv
      * width: 4
      * #verilog_aval_bval: verilog_unsignedbv
  * value: 7[4:2]) ? 1 : main.z)

  next(main.cnt1) == main.cnt1_aux0



Latch dependencies. varmap
--smt2 dumps the SMT2 formula to a file

https://www.cprover.org/ebmc/manual/tutorial/

In [60]:
%%file /tmp/main.sv
module my_add(input a, b, output [1:0] y);

  assign y[0]=a^b;
  assign y[1]=a&b;

endmodule

module main(input a, b);

  wire [1:0] result;

  my_add adder(a, b, result);
  // wire q;
  // assign q=a + b;

  //p1: assert a + b == result[0];
  p1 : assert property (a == result[0]);

endmodule

Overwriting /tmp/main.sv


In [63]:
! yosys -p "read_verilog  -sv /tmp/main.sv; prep -top main; write_smt2 -wires -mem -bv /tmp/main.smt2"


 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42+40 (git sha1 a739e21a5, clang++ 14.0.0-1ubuntu1.1 -fPIC -Os)

-- Running command `read_verilog  -sv /tmp/main.sv; prep -top main; write_smt2 -wires -mem -bv /tmp/main.smt2' --

1. Executing Verilog-2005 frontend: /tmp/main.sv
Parsing SystemVerilog input from `/tmp/main.sv' to AST representation.
Generating RTLIL representation for module `\my_add'.
Generating RTLIL representation for module `\main'.
Successfully finished Verilog frontend.

2. Executing PREP pass.

2.1. Executing HIERARCHY pass (managing design hierarchy).

2.1.1. Analyzing design hierarchy..
Top module:  \main
Use

yosys bmc takes an smt file.
sby needs some kind of other file.



In [64]:
! yosys-smtbmc -s z3 /tmp/main.smt2

##   0:00:00  Solver: z3
##   0:00:00  Checking assertions in step 0..
##   0:00:00  Checking assertions in step 1..
##   0:00:00  Checking assertions in step 2..
##   0:00:00  Checking assertions in step 3..
##   0:00:00  Checking assertions in step 4..
##   0:00:00  Checking assertions in step 5..
##   0:00:00  Checking assertions in step 6..
##   0:00:00  Checking assertions in step 7..
##   0:00:00  Checking assertions in step 8..
##   0:00:00  Checking assertions in step 9..
##   0:00:00  Checking assertions in step 10..
##   0:00:00  Checking assertions in step 11..
##   0:00:00  Checking assertions in step 12..
##   0:00:00  Checking assertions in step 13..
##   0:00:00  Checking assertions in step 14..
##   0:00:00  Checking assertions in step 15..
##   0:00:00  Checking assertions in step 16..
##   0:00:00  Checking assertions in step 17..
##   0:00:00  Checking assertions in step 18..
##   0:00:00  Checking assertions in step 19..
##   0:00:00  Status: PASSED



 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42+40 (git sha1 a739e21a5, clang++ 14.0.0-1ubuntu1.1 -fPIC -Os)

-- Running command `read_verilog -formal -sv /tmp/main.sv; prep -top main;' --

1. Executing Verilog-2005 frontend: /tmp/main.sv
Parsing formal SystemVerilog input from `/tmp/main.sv' to AST representation.
/tmp/main.sv:16: ERROR: syntax error, unexpected TOK_ID, expecting TOK_PROPERTY


In [34]:
! sby -t /tmp/main.sv

ERROR: Sby file does not have .sby file extension.


In [19]:
! ebmc --module main /tmp/main.sv --waveform

Parsing /tmp/main.sv
Converting
Type-checking Verilog::my_add
Type-checking Verilog::main
using default bound 1
Generating Decision Problem
Using MiniSAT 2.2.1 with simplifier
Properties
Solving with propositional reduction
Checking main.assert.1
SAT checker: instance is UNSATISFIABLE
UNSAT: No counterexample found within bound

** Results:
[main.assert.1] always main.a + main.b == main.result[0]: PROVED up to bound 1


In [20]:
%%file /tmp/counter.sv
module counter(
  output [7:0] out,
  input enable, input clk);

  reg [7:0] state;
  assign out = state;

  initial state = 0;

  always @(posedge clk)
    if(enable)
      state = state + 1;

  assert property (enable && state[0] |-> ##1 !state[0]);
  assert property (state!=3);

endmodule

Writing /tmp/counter.sv


In [21]:
! ebmc --module counter /tmp/counter.sv --waveform --bound 10

Parsing /tmp/counter.sv
Converting
Type-checking Verilog::counter
Generating Decision Problem
Using MiniSAT 2.2.1 with simplifier
Properties
Solving with propositional reduction
Checking counter.assert.1
SAT checker: instance is UNSATISFIABLE
UNSAT: No counterexample found within bound
Checking counter.assert.2
SAT checker: instance is SATISFIABLE
SAT: counterexample found

** Results:
[counter.assert.1] always (counter.enable && counter.state[0] |-> (##1 !counter.state[0])): PROVED up to bound 10
[counter.assert.2] always counter.state != 3: REFUTED
Counterexample:
                0  1  2  3  4  5  6  7  8  9 10
   counter.clk                                 
counter.enable  1  1  1  0  0  0  0  0  0  0  0
   counter.out  0  1  2  3  3  3  3  3  3  3  3
 counter.state  0  1  2  3  3  3  3  3  3  3  3


# RiscV

Serv core - smallest riscv? https://github.com/olofk/serv.git  LUT FF flip flop
https://github.com/enjoy-digital/litex litex supports it

https://github.com/YosysHQ/picorv32
https://github.com/BrunoLevy/learn-fpga
femtorv quARK https://github.com/BrunoLevy/learn-fpga/blob/master/FemtoRV/RTL/PROCESSOR/femtorv32_quark.v

https://github.com/rust-embedded/riscv low level access to risc v processor in rust
https://popovicu.com/posts/bare-metal-rust-risc-v-with-dynamic-memory/
Kani + aeneas? https://github.com/AeneasVerif/aeneas


https://crates.io/crates/rrs-lib
https://crates.io/crates/riscv
https://crates.io/crates/rvemu less stars

https://github.com/black-parrot/black-parrot  "Other mappings, such the multiplier to DSP48, could be improved." Something zach mentioned in a mocha ideation m,eeting
https://github.com/chipsalliance/dromajo riscv simulator for rtl cosim
https://bellard.org/tinyemu/ 


In [11]:
%%file /tmp/risc.s
.global start
_start:
    mov r0, #1
    mov r1, #2
    add r2, r0, r1
    mov r3, #0
    swi 0


Overwriting /tmp/risc.s


In [3]:
! riscv64-linux-gnu-as -o /tmp/risc.o /tmp/risc.s

/bin/bash: line 1: riscv64-linux-gnu-as: command not found


In [12]:
! ~/Downloads/riscv_toolchain/bin/riscv32-unknown-elf-as -o /tmp/risc.o /tmp/risc.s

In [ ]:
%%file /tmp/rustemu.rs
use rrs_lib::{HartState, MemAccessSize, Memory};
use rrs_lib::memories::VecMemory;
use rrs_lib::instruction_executor::{InstructionExecutor, InstructionException};
use rrs_lib::instruction_string_outputter::InstructionStringOutputter;

fn simulate_riscv() {
  let mut hart = HartState::new();
  // Memory contains these instructions:
  // lui x2, 0x1234b
  // lui x3, 0xf387e
  // add x1, x2, x3
  let mut mem = VecMemory::new(vec![0x1234b137, 0xf387e1b7, 0x003100b3]);

  hart.pc = 0;

  // InstructionExecutor implements IntructionProcessor. The step function calls
  // process_instruction internally and handles things like updating the PC.
  let mut executor = InstructionExecutor {
      hart_state: &mut hart,
      mem: &mut mem,
  };

  // Execute first instruction
  output_disass(&mut executor);
  assert_eq!(executor.step(), Ok(()));
  assert_eq!(executor.hart_state.registers[2], 0x1234b000);

  // Execute second instruction
  output_disass(&mut executor);
  assert_eq!(executor.step(), Ok(()));
  assert_eq!(executor.hart_state.registers[3], 0xf387e000);

  // Execute third instruction
  output_disass(&mut executor);
  assert_eq!(executor.step(), Ok(()));
  assert_eq!(executor.hart_state.registers[1], 0x05bc9000);

  // Memory only contains three instructions so next step will produce a fetch error
  assert_eq!(executor.step(), Err(InstructionException::FetchError(0xc)));
}

fn output_disass<M: Memory>(executor: &mut InstructionExecutor<M>) {
  let mut outputter = InstructionStringOutputter { insn_pc: executor.hart_state.pc };
  let insn_bits = executor.mem.read_mem(executor.hart_state.pc, MemAccessSize::Word).unwrap();
  println!("{}", rrs_lib::process_instruction(&mut outputter, insn_bits).unwrap());
}

# Interrupts
https://www.cs.ox.ac.uk/tom.melham/pub/Kroening-2015-EVL.pdf cbmc

goto-instrument --isr interrupt_1 --dump-c

https://users.cs.utah.edu/~regehr/papers/tv06.pdf Regehr

https://sos-vo.org/sites/sos-vo.org/files/sos_files/Interrupts_in_OS_code_lets_reason_about_them_Yes_this_means_concurrency.pdf

In [1]:
%%file /tmp/int1.c
#include <assert.h>

int x = 0;

void interrupt_1() {
    x += 1;
}

int main (int argc, char *argv[]) {
    x = 0;
    assert(x == 0);
    return x;
}

Writing /tmp/int1.c


In [ ]:
! c

# Bootloading
https://projekter.aau.dk/projekter/files/422795280/P10__23_.pdf  Formally Verifying the Correctness and Safety of OpenTitan
Boot Code using CBMC
Jacob Gosch Søndergaard,
Kristoffer Skagbæk Jensen
Spring 2021

Kim Guldstrand Larsen https://dblp.org/pid/l/KimGuldstrandLarsen.html
https://vbn.aau.dk/en/publications/a-modeling-concept-for-formal-verification-of-os-based-compositio
A Modeling Concept for Formal Verification of OS-Based Compositional Software  https://link.springer.com/chapter/10.1007/978-3-031-30826-0_2

https://link.springer.com/chapter/10.1007/978-3-031-49252-5_12  Comparative Analysis of Uppaal  SMC, ns-3 and MATLAB/Simulink

https://uppaal.org/documentation/
https://docs.uppaal.org/

https://projekter.aau.dk/projekter/files/422795285/P10__24_.pdf Formally Verifying Security Properties for
OpenTitan Boot Code with Uppaal

https://github.com/KVISDAOWNER/b00t-c0d3
https://github.com/Tutter/OpenTitan-Formal-Verification


] Bjarke Hilmer Møller, Magnus Winkel Pedersen, and Bøgedal Tobias Worm. “Formally Verifying Security
Properties for OpenTitan Boot Code with UPPAAL”. To Appear. MA thesis. AAU, 2021.

https://news.ycombinator.com/item?id=41161580  	A cryptographically secure bootloader for RISC-V in Rust (codethink.co.uk)
https://www.codethink.co.uk/articles/2024/secure_bootloader/ 
riscv vector cryptography extension

I have old notes somewhere

PUF https://en.wikipedia.org/wiki/Physical_unclonable_function

https://arstechnica.com/security/2024/07/secure-boot-is-completely-compromised-on-200-models-from-5-big-device-makers/

https://github.com/vvaltchev/tilck
http://github.com/oreboot/oreboot

uboot


https://github.com/river-li/awesome-uefi-security

open titan 
https://github.com/lowRISC/opentitan.git
https://opentitan.org/book/doc/security/specs/attestation/ hardware attestation
https://opentitan.org/book/sw/device/silicon_creator/rom/index.html ROM is first stage boot

https://docs.arduino.cc/retired/hacking/software/Bootloader/ arduino bootloader

Running risc v. Emulating m,icrocontroller?


privlenged architexcture riscv
https://riscv.org/wp-content/uploads/2017/05/riscv-privileged-v1.10.pdf

opensbi https://github.com/riscv-software-src/opensbi

acronyms out the wing wong
PMP - physical memory protection
TPM

PCR registers. Extend and hash. Makes a registery merkle list thing.

bootkeeper https://arxiv.org/pdf/1903.12505

[1] Oleksandr Bazhaniuk, John Loucaides, Lee Rosenbaum, Mark R Tuttle, and Vincent Zimmer. 2015. Symbolic execution for BIOS security. In 9th USENIX Workshop
on Offensive Technologies (WOOT ’15).
[2] Jonathan Brossard. 2012. Hardware backdooring is practical. BlackHat, Las Vegas,
USA (2012).
[3] Danilo Bruschi, Lorenzo Cavallaro, Andrea Lanzi, and Mattia Monga. 2005. Replay attack in TCG specification and solution. In Proceeding of the 21st Annual
Computer Security Applications Conference (ACSAC ’05). IEEE, 127–137.
[4] John Butterworth, Corey Kallenberg, Xeno Kovah, and Amy Herzog. 2013. BIOS
Chronomancy: Fixing the core root of trust for measurement. In Proceedings of
the 2013 ACM SIGSAC conference on Computer & Communications Security (CCS
’13). ACM, 25–36.


https://wiki.archlinux.org/title/Trusted_Platform_Module linux access to tpm

https://surface.syr.edu/cgi/viewcontent.cgi?article=1182&context=eecs_techreports sable constable. isabelle verified bootloader
skinit instruction. Start process with hash in a pcr and completely fresh isolated? What's the point though? If the insecure part has any effect. If it doesn't why was it there?
Dynamic

https://google.github.io/tpm-js/ javascript tpm in browser

## Attacks and Glitching and Fault injection

Chipwhisperer http://wiki.newae.com/V4:Tutorial_A2_Introduction_to_Glitch_Attacks_(including_Glitch_Explorer)

https://www.youtube.com/watch?v=XvGcQgx9Jg8&ab_channel=BlackHat breaking secure bootloaders blackhat - bootloader has a bad parser in it

https://limitedresults.com/2019/09/pwn-the-esp32-secure-boot/

Glitching / fault injection. Look at code, correlated power draw? Wiggle voltage at opporuitne times. Or lasers if you're fancy

https://github.com/janvdherrewegen/bootl-attacks
https://www.youtube.com/watch?v=T1NHrzhtqzA&ab_channel=IACR

https://machiry.github.io/files/glitchresistor.pdf glitch resistor. Modellign glitching and llvm plugin for resistance
Redundant checks because you aren't sure insturctions weren't skipped.

https://static.linaro.org/connect/lvc21f/presentations/LVC21F-116.pdf
https://ieeexplore.ieee.org/document/8167709
https://ibex-core.readthedocs.io/en/latest/03_reference/security.html

https://github.com/ucsb-seclab/BootStomp
https://dl.acm.org/doi/10.5555/3241189.3241251

https://www.riscure.com/publication/fault-mitigation-patterns/

## AB Update
"A/B" update https://opentitan.org/book/doc/security/logical_security_model/index.html?highlight=A%2FB#firmware-update-considerations So the stage0 bootloader is in charge of checking the sanctity of the new bootimage and picking the newest one

https://source.android.com/docs/core/ota/ab


https://source.android.com/docs/core/ota/ab/ab_implement

https://bootlin.com/pub/conferences/2022/elce/opdenacker-implementing-A-B-system-u[…]boot/opdenacker-implementing-A-B-system-updates-with-u-boot.pdf

Signature
decryption
other things?
rboot https://github.com/raburton/rboot/blob/master/rboot.c


# Qemu
sudo apt-get update
sudo apt-get install opensbi qemu-system-misc u-boot-qemu
https://www.qemu.org/docs/master/system/target-riscv.html

https://popovicu.com/posts/bare-metal-programming-risc-v/

QEMU -bios flag
0x80000000
qemu loads up opensbi

https://github.com/qemu/qemu/blob/master/hw/riscv/virt.c#L78-L99 memory map

```C
static const MemMapEntry virt_memmap[] = {
    [VIRT_DEBUG] =        {        0x0,         0x100 },
    [VIRT_MROM] =         {     0x1000,        0xf000 },
    [VIRT_TEST] =         {   0x100000,        0x1000 },
    [VIRT_RTC] =          {   0x101000,        0x1000 },
    [VIRT_CLINT] =        {  0x2000000,       0x10000 },
    [VIRT_ACLINT_SSWI] =  {  0x2F00000,        0x4000 },
    [VIRT_PCIE_PIO] =     {  0x3000000,       0x10000 },
    [VIRT_PLATFORM_BUS] = {  0x4000000,     0x2000000 },
    [VIRT_PLIC] =         {  0xc000000, VIRT_PLIC_SIZE(VIRT_CPUS_MAX * 2) },
    [VIRT_APLIC_M] =      {  0xc000000, APLIC_SIZE(VIRT_CPUS_MAX) },
    [VIRT_APLIC_S] =      {  0xd000000, APLIC_SIZE(VIRT_CPUS_MAX) },
    [VIRT_UART0] =        { 0x10000000,         0x100 },
    [VIRT_VIRTIO] =       { 0x10001000,        0x1000 },
    [VIRT_FW_CFG] =       { 0x10100000,          0x18 },
    [VIRT_FLASH] =        { 0x20000000,     0x4000000 },
    [VIRT_IMSIC_M] =      { 0x24000000, VIRT_IMSIC_MAX_SIZE },
    [VIRT_IMSIC_S] =      { 0x28000000, VIRT_IMSIC_MAX_SIZE },
    [VIRT_PCIE_ECAM] =    { 0x30000000,    0x10000000 },
    [VIRT_PCIE_MMIO] =    { 0x40000000,    0x40000000 },
    [VIRT_DRAM] =         { 0x80000000,           0x0 },
};

/* PCIe high mmio is fixed for RV32 */
#define VIRT32_HIGH_PCIE_MMIO_BASE  0x300000000ULL
#define VIRT32_HIGH_PCIE_MMIO_SIZE  (4 * GiB)

/* PCIe high mmio for RV64, size is fixed but base depends on top of RAM */
#define VIRT64_HIGH_PCIE_MMIO_SIZE  (16 * GiB)
```

In [ ]:
%%file /tmp/

In [ ]:
%%bash
riscv32-none-elf-as -o /tmp/int1.o /tmp/int1.c
riscv32-none-elf-ld -o /tmp/int1 /tmp/int1.o
qemu-system-riscv32 -nographic -machine spike_v1.10 -kernel /tmp/int1

In [2]:
c!qemu-system-riscv32

qemu-system-riscv32: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE


In [17]:
%%file /tmp/minimal.ld
MEMORY {
    dram_space (rwx): ORIGIN = 0x80000000, LENGTH = 0x128
}

SECTIONS {
    .text : {
        minimal.o(.text.minimal)
    } > dram_space
    }

Overwriting /tmp/minimal.ld


In [16]:
%%file /tmp/minimal.s
.global _start
.section .text.bios

_start: 
    addi a0, x0, 0x68
    li a1, 0x10000000
    sb a0, (a1) #'h'

loop: j loop


Overwriting /tmp/minimal.s


In [25]:
%%bash
PATH=~/Downloads/riscv_toolchain/bin:$PATH
riscv32-unknown-elf-as -o /tmp/minimal.o /tmp/minimal.s
#riscv32-unknown-elf-objdump -d /tmp/minimal.o
riscv32-unknown-elf-ld -T /tmp/minimal.ld -o /tmp/minimal /tmp/minimal.o
#qemu-system-riscv32 -nographic -machine spike_v1.10 -bios /tmp/minimal

riscv32-unknown-elf-ld: cannot find minimal.o: No such file or directory


CalledProcessError: Command 'b'PATH=~/Downloads/riscv_toolchain/bin:$PATH\nriscv32-unknown-elf-as -o /tmp/minimal.o /tmp/minimal.s\n#riscv32-unknown-elf-objdump -d /tmp/minimal.o\nriscv32-unknown-elf-ld -T /tmp/minimal.ld -o /tmp/minimal /tmp/minimal.o\n#qemu-system-riscv32 -nographic -machine spike_v1.10 -bios /tmp/minimal\n'' returned non-zero exit status 1.

In [22]:
! ~/Downloads/riscv_toolchain/bin/objdump -d /tmp/minimal.o

/bin/bash: line 1: /home/philip/Downloads/riscv_toolchain/bin/objdump: No such file or directory


## Esp32
https://docs.espressif.com/projects/esp-idf/en/latest/esp32/get-started/linux-macos-setup.html#get-started-get-esp-idf
vscode esp-idf


```
cd ~/esp
cp -r $IDF_PATH/examples/get-started/hello_world .
cd ~/esp/hello_world
idf.py set-target esp32
idf.py menuconfig
idf.py build
idf.py -p PORT flash
```

https://github.com/cjacker/opensource-toolchain-esp32
putting stuff in `~/.expressif`


In [ ]:
%%file /tmp/minimal.s

.global _start
.section .text.bios



### esptool
https://docs.espressif.com/projects/esptool/en/latest/esp32/esptool/index.html

In [5]:
! esptool.py -p /dev/ttyACM1 chip_id  # it'll scan ports

esptool.py v4.8.1
Serial port /dev/ttyACM1
Connecting...
Detecting chip type... ESP32-C6
Chip is ESP32-C6 (QFN40) (revision v0.0)
Features: WiFi 6, BT 5, IEEE802.15.4
Crystal is 40MHz
MAC: 40:4c:ca:ff:fe:4e:6b:a0
BASE MAC: 40:4c:ca:4e:6b:a0
MAC_EXT: ff:fe
Uploading stub...
Running stub...
Stub running...
MAC: 40:4c:ca:ff:fe:4e:6b:a0
BASE MAC: 40:4c:ca:4e:6b:a0
MAC_EXT: ff:fe
Hard resetting via RTS pin...


 amazon bedrock https://aws.amazon.com/bedrock/claude/


# Userland load

In [25]:
%%file /tmp/code.s
.global _start

_start:
    mov $42, %rax
    ret

Writing /tmp/code.s


In [28]:
%%bash
as -o /tmp/code.o /tmp/code.s
objcopy -O binary /tmp/code.o /tmp/code.bin
hexdump -v -e '"0x" 1/1 "%02X" ", "' /tmp/code.bin

0x48, 0xC7, 0xC0, 0x2A, 0x00, 0x00, 0x00, 0xC3, 

In [21]:
%%file /tmp/simploader.c
#include <stdio.h>
#include <stdint.h>

unsigned char code[] __attribute__ ((section (".text"))) = {
    0x48, 0xC7, 0xC0, 0x2A, 0x00, 0x00, 0x00, 0xC3
};
int main(){
    int (*func)() = (int (*)())code;
    int retcode = func();
    
    printf("retcode %d\n", retcode);
    return 0;
}

Overwriting /tmp/simploader.c


In [25]:
!frama-c -wp /tmp/simploader.c

[kernel] Parsing /tmp/simploader.c (with preprocessing)
[kernel:typing:incompatible-pointer-types] /tmp/simploader.c:8: Warning: 
  casting function from unsigned char *
[kernel:annot:missing-spec] FRAMAC_SHARE/libc/stdio.h:211: Warning: 
  Neither code nor explicit exits and terminates for function printf_va_1,
   generating default clauses. See -generated-spec-* options for more info
[wp] /tmp/simploader.c:7: Warning: 
  In 'main', no 'calls' specification for statement(s) on line(s): line 9, 
  Assuming that they can call 'main'
[wp] Warning: Missing RTE guards
[wp] /tmp/simploader.c:9: Warning: Missing 'calls' for default behavior
[wp] /tmp/simploader.c:9: Warning: 
  Unknown callee, considering non-terminating call
[wp] /tmp/simploader.c:9: Warning: 
  Missing decreases clause on recursive function main, call must be unreachable
[wp] /tmp/simploader.c:8: Warning: 
  Cast with incompatible pointers types (source: uint8*) (target: void**)
[wp] User Error: Prover 'Alt-Ergo' not found

In [20]:
%%file /tmp/simploader.c
#include <stdio.h>
#include <stdint.h>
#include <sys/mman.h>
#include <string.h>
#include <unistd.h>

unsigned char code[] = {
    //0x90, 0x90, 0xC3 /// (NOP; NOP; RET)
    0x48, 0xC7, 0xC0, 0x2A, 0x00, 0x00, 0x00, 0xC3
    // Put code here
};
int main(){
    size_t pagesize = sysconf(_SC_PAGESIZE); //getpagesize();
    void* exec_mem = mmap(NULL, pagesize, PROT_WRITE | PROT_EXEC, MAP_ANON | MAP_PRIVATE, -1, 0); // Hmm. shouldn't cbmc have noted I should have checked for fail here?
    memcpy(exec_mem, code, sizeof(code));

    int (*func)() = (int (*)())exec_mem;
    int retcode = func();
    
    printf("retcode %d\n", retcode);
    munmap(exec_mem, pagesize);
    return 0;
}

Writing /tmp/simploader.c


In [44]:
!gcc -Wall -Wextra -Werror -o /tmp/simploader /tmp/simploader.c && /tmp/simploader

/tmp/ccXxw4Nh.s: Assembler messages:
/tmp/ccXxw4Nh.s:4: Warning: ignoring changed section attributes for .text
retcode 42


In [ ]:
!

In [35]:
!cbmc /tmp/simploader.c --pointer-check

CBMC version 6.0.1 (cbmc-6.0.1-5-g54c20cdb91) 64-bit x86_64 linux
Type-checking simploader
Generating GOTO Program
Adding CPROVER library (x86_64)


Removal of function pointers and virtual functions
Generic Property Instrumentation
Starting Bounded Model Checking
Passing problem to propositional reduction
converting SSA
Running propositional reduction
SAT checker: instance is SATISFIABLE
Running propositional reduction
SAT checker inconsistent: instance is UNSATISFIABLE

** Results:
/tmp/simploader.c function main
[main.precondition_instance.1] line 16 memcpy src/dst overlap: SUCCESS
[main.precondition_instance.2] line 16 memcpy source region readable: SUCCESS
[main.precondition_instance.3] line 16 memcpy destination region writeable: FAILURE
[main.pointer_dereference.1] line 19 dereference failure: pointer NULL in func: SUCCESS
[main.pointer_dereference.2] line 19 dereference failure: pointer invalid in func: SUCCESS
[main.pointer_dereference.3] line 19 dereference failure: deallocated dynamic object in func: SUCCESS
[main.pointer_dereference.4] line 19 dereference failure: dead object in func: SUCCESS
[main.pointer_dereference.5

In [ ]:
%%file /tmp/simploader.c

int code[] = {
    // Put code here
}
// int codes[3][1024] = {{},{}, {}}
int crc = 0xdeadbeef;
int pub_auth_key;

int auth(int* code, int size, int key ){

}

int decrypt_buffer[1024];
int decrypt(int , int size, int* buffer){

}

typedef struct config_t{
} config_t;

int main(){
    check_config();
    auth(code, sizeof(code), sig);
    decrypt(code, sizeof(code), decrypt_buffer);
    (void* code())();
}



In [ ]:
!cbmc /tmp/simploader.c

# Simulation
cocotb
verilator https://verilator.org/guide/latest/install.html#git-install
ibex verilator https://github.com/lowRISC/ibex/blob/master/examples/simple_system/README.md


Havbing the ability to external call from inside of verilog is how you can maybe stub out peripherals.

PLI https://en.wikipedia.org/wiki/Verilog_Procedural_Interface
https://www.asic-world.com/verilog/pli.html

VPI (pli 2)


DPI https://en.wikipedia.org/wiki/SystemVerilog_DPI direct programming interface


## icarus
https://steveicarus.github.io/iverilog/usage/vpi.html
Principles of Verilog PLI by Swapnajit Mittra. ISBN 0-7923-8477-6

The Verilog PLI Handbook by Stuart Sutherland. ISBN 0-7923-8489-X

vlog_startup routines is at able of function pointers called

```
% gcc -c -fpic hello.c
% gcc -shared -o hello.vpi hello.o -lvpi
```

In [7]:
%%file /tmp/hello.c

# include  <vpi_user.h>

static int hello_compiletf(char*user_data)
{
      return 0;
}

static int hello_calltf(char*user_data)
{
      vpi_printf("Hello, Buddy!\n");
      return 0;
}

void hello_register()
{
      s_vpi_systf_data tf_data;

      tf_data.type      = vpiSysTask;
      tf_data.tfname    = "$hello";
      tf_data.calltf    = hello_calltf;
      tf_data.compiletf = hello_compiletf;
      tf_data.sizetf    = 0;
      tf_data.user_data = 0;
      vpi_register_systf(&tf_data);
}

void (*vlog_startup_routines[])() = {
    hello_register,
    0
};

Overwriting /tmp/hello.c


In [8]:
%%file /tmp/hello.v
module main;
  initial $hello;
endmodule


Overwriting /tmp/hello.v


In [9]:
%%bash
cd /tmp
iverilog-vpi hello.c # gcc wrapper 
iverilog -ohello.vvp hello.v
vvp -M. -mhello hello.vvp

Compiling hello.c...


hello.c: In function ‘hello_compiletf’:
hello.c:4:33: warning: unused parameter ‘user_data’ [-Wunused-parameter]
    4 | static int hello_compiletf(char*user_data)
      |                            ~~~~~^~~~~~~~~
hello.c: In function ‘hello_calltf’:
hello.c:9:30: warning: unused parameter ‘user_data’ [-Wunused-parameter]
    9 | static int hello_calltf(char*user_data)
      |                         ~~~~~^~~~~~~~~
hello.c: At top level:
hello.c:15:6: warning: function declaration isn’t a prototype [-Wstrict-prototypes]
   15 | void hello_register()
      |      ^~~~~~~~~~~~~~
hello.c:28:1: warning: function declaration isn’t a prototype [-Wstrict-prototypes]
   28 | void (*vlog_startup_routines[])() = {
      | ^~~~


Making hello.vpi from  hello.o...
Hello, Buddy!


## verilator
So there are things called dpi which let you call C functions from within verilator code. That's how you can hook your simulated guy to system serial device.
Or ports.
Does iverilog let you do this
https://projectf.io/posts/verilog-sim-verilator-sdl/

https://tomverbeure.github.io/2020/08/08/CXXRTL-the-New-Yosys-Simulation-Backend.html



# FPGA

https://pulp-platform.org/docs/coscup2024/phsauter_coscup24.pdf  Designing Linux-capable systems using open EDA tools Pulp
https://github.com/pulp-platform
Used for ETH Zürich VLSI 2 lecture starting 2025 Croc soc
github.com/pulp-platform/croc

place and route


https://github.com/chipsalliance/Surelog  SystemVerilog 2017 Pre-processor, Parser, Elaborator, UHDM Compiler. Provides IEEE Design/TB C/C++ VPI and Python AST API.
https://github.com/bespoke-silicon-group/bsg_sv2v/tree/master sysverilog to verilog
https://github.com/chipsalliance/rocket-chip

olof kindgren
fusesoc https://github.com/olofk/fusesoc package manager 
edalizer - port acorss different guys? https://github.com/olofk/edalize


iverilog

https://projectf.io/posts/verilog-sim-verilator-sdl/ verilator to sdl


Lakeroad / sam coward things
It is an interesting and possibly noble endeavor to try and make open hardware / fpga compiling as good as open software

Get out some intermediate rwepresentation from yosys.
Emulate stuff via iverilog
nmigen

https://github.com/YosysHQ/oss-cad-suite-build#installation
https://yowasp.org/ pip install yowasp-yosys

ffirtl https://github.com/chipsalliance/firrtl

RTLIL is yosys internal IR. Text rep

Lakeroad https://arxiv.org/abs/2401.16526
https://github.com/uwsampl/lakeroad
https://github.com/uwsampl/lakeroad/blob/main/bin/verilog_to_racket.py yosys btor output


Could also root around in rachit's stuff

https://github.com/f4pga/f4pga-arch-defs

In [6]:
%%file /tmp/counter.v
module counter(
    input wire clk,
    input wire rst,
    output reg [7:0] count
);

always @(posedge clk or posedge rst) begin
    if (rst) begin
        count <= 8'b0;
    end else begin
        count <= count + 1;
    end
end

endmodule

Overwriting /tmp/counter.v


In [8]:
%%bash
#output btor2
# ERROR: Unsupported cell type $_DFF_PP0_ for cell counter.$auto$ff.cc:266:slice$89 -- please run `async2sync; dffunmap` or `clk2fflogic` before `write_btor`.
yosys -p 'read_verilog /tmp/counter.v; synth -top counter; async2sync; dffunmap; write_btor /tmp/counter.btor'


 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42+40 (git sha1 a739e21a5, clang++ 14.0.0-1ubuntu1.1 -fPIC -Os)

-- Running command `read_verilog /tmp/counter.v; synth -top counter; async2sync; dffunmap; write_btor /tmp/counter.btor' --

1. Executing Verilog-2005 frontend: /tmp/counter.v
Parsing Verilog input from `/tmp/counter.v' to AST representation.
Generating RTLIL representation for module `\counter'.
Successfully finished Verilog frontend.

2. Executing SYNTH pass.

2.1. Executing HIERARCHY pass (managing design hierarchy).

2.1.1. Analyzing design hierarchy..
Top module:  \counter

2.1.2. Analyzing design hierarchy..
Top m


2.6.4. Executing OPT_REDUCE pass (consolidate $*mux and $reduce_* inputs).
  Optimizing cells in module \counter.
Performed a total of 0 changes.

2.6.5. Executing OPT_MERGE pass (detect identical cells).
Finding identical cells in module `\counter'.
Removed a total of 0 cells.

2.6.6. Executing OPT_DFF pass (perform DFF optimizations).

2.6.7. Executing OPT_CLEAN pass (remove unused cells and wires).
Finding unused cells or wires in module \counter..

2.6.8. Executing OPT_EXPR pass (perform const folding).
Optimizing module counter.

2.6.9. Finished OPT passes. (There is nothing left to do.)

2.7. Executing FSM pass (extract and optimize FSM).

2.7.1. Executing FSM_DETECT pass (finding FSMs in design).

2.7.2. Executing FSM_EXTRACT pass (extracting FSM from design).

2.7.3. Executing FSM_OPT pass (simple optimizations of FSMs).

2.7.4. Executing OPT_CLEAN pass (remove unused cells and wires).
Finding unused cells or wires in module \counter..

2.7.5. Executing FSM_OPT pass (simple op

In [19]:
import boolector

ModuleNotFoundError: No module named 'boolector'

In [18]:
import bitwuzla
tm = bitwuzla.TermManager()
options = bitwuzla.Options()
p = bitwuzla.Parser(tm, options, "btor2")
p.parse('/tmp/counter.btor')

BitwuzlaException: model checking extensions not supported

In [12]:
%%file /tmp/cpu_test.v

module cpu_test;
    reg clk;
    reg reset;
    wire [7:0] output_signal;

    // Instantiate your CPU module
    /*
    cpu my_cpu (
        .clk(clk),
        .reset(reset),
        .output_signal(output_signal)
    ); */
    assign output_signal = 8'b10101010;

    initial begin
        clk = 0;
        reset = 1;
        #5 reset = 0;
        #1000 $finish;
    end

    always #5 clk = ~clk;

    always @(posedge clk) begin
        // Simple example of ASCII visualization
        $display("Cycle %d: Output signal = %b", $time, output_signal);
    end
endmodule


Overwriting /tmp/cpu_test.v


In [14]:
%%bash
iverilog -o /tmp/cpu_test.vvp /tmp/cpu_test.v
vvp /tmp/cpu_test.vvp

Cycle                    5: Output signal = 10101010
Cycle                   15: Output signal = 10101010
Cycle                   25: Output signal = 10101010
Cycle                   35: Output signal = 10101010
Cycle                   45: Output signal = 10101010
Cycle                   55: Output signal = 10101010
Cycle                   65: Output signal = 10101010
Cycle                   75: Output signal = 10101010
Cycle                   85: Output signal = 10101010
Cycle                   95: Output signal = 10101010
Cycle                  105: Output signal = 10101010
Cycle                  115: Output signal = 10101010
Cycle                  125: Output signal = 10101010
Cycle                  135: Output signal = 10101010
Cycle                  145: Output signal = 10101010
Cycle                  155: Output signal = 10101010
Cycle                  165: Output signal = 10101010
Cycle                  175: Output signal = 10101010
Cycle                  185: Output signal = 10

https://github.com/cocotb/cocotb

Oh. cocotb ius like a testbench. It uses icarus. Hmm. but maybe its an interface to icarus

In [10]:
%%file /tmp/LUT2.v
// What does this do?
`default_nettype none
module LUT2(input A, B, output Z);
    parameter [3:0] init = 4'h0;
    wire [1:0] s1 = B ?     init[ 3:2] :     init[1:0];
    assign Z =      A ?          s1[1] :         s1[0];
endmodule

Overwriting /tmp/LUT2.v


In [6]:
%%file /tmp/yoscript.ys
read_verilog /tmp/LUT2.v

Overwriting /tmp/yoscript.ys


In [2]:
! yowasp-yosys -p read_verilog /tmp/LUT2.v #-s /tmp/yoscript.ys


 /----------------------------------------------------------------------------\
 |  yosys -- Yosys Open SYnthesis Suite                                       |
 |  Copyright (C) 2012 - 2024  Claire Xenia Wolf <claire@yosyshq.com>         |
 |  Distributed under an ISC-like license, type "license" to see terms        |
 \----------------------------------------------------------------------------/
 Yosys 0.42 (git sha1 9b6afcf3f, ccache clang 14.0.0-1ubuntu1.1 -Os -flto -flto)

-- Parsing `/tmp/LUT2.v' using frontend ` -vlog2k' --
ERROR: Can't open input file `/tmp/LUT2.v' for reading: No such file or directory


hmm. Looks like boolector or bitwuzla python bindings might support parsing btor2
https://bitwuzla.github.io/docs/python/interface.html#bitwuzla.Parser.parse
https://boolector.github.io/docs/boolector.html#pyboolector.Boolector.Parse

In [12]:
%%file /tmp/fifo.v

// address generator/counter
module addr_gen 
#(  parameter MAX_DATA=256,
	localparam AWIDTH = $clog2(MAX_DATA)
) ( input en, clk, rst,
	output reg [AWIDTH-1:0] addr
);
	initial addr <= 0;

	// async reset
	// increment address when enabled
	always @(posedge clk or posedge rst)
		if (rst)
			addr <= 0;
		else if (en) begin
			if (addr == MAX_DATA-1)
				addr <= 0;
			else
				addr <= addr + 1;
		end
endmodule //addr_gen

// Define our top level fifo entity
module fifo 
#(  parameter MAX_DATA=256,
	localparam AWIDTH = $clog2(MAX_DATA)
) ( input wen, ren, clk, rst,
	input [7:0] wdata,
	output reg [7:0] rdata,
	output reg [AWIDTH:0] count
);
	// fifo storage
	// sync read before write
	wire [AWIDTH-1:0] waddr, raddr;
	reg [7:0] data [MAX_DATA-1:0];
	always @(posedge clk) begin
		if (wen)
			data[waddr] <= wdata;
		rdata <= data[raddr];
	end // storage

	// addr_gen for both write and read addresses
	addr_gen #(.MAX_DATA(MAX_DATA))
	fifo_writer (
		.en     (wen),
		.clk    (clk),
		.rst    (rst),
		.addr   (waddr)
	);

	addr_gen #(.MAX_DATA(MAX_DATA))
	fifo_reader (
		.en     (ren),
		.clk    (clk),
		.rst    (rst),
		.addr   (raddr)
	);

	// status signals
	initial count <= 0;

	always @(posedge clk or posedge rst) begin
		if (rst)
			count <= 0;
		else if (wen && !ren)
			count <= count + 1;
		else if (ren && !wen)
			count <= count - 1;
	end

endmodule

Writing /tmp/fifo.v


In [2]:
! yosys /tmp/fifo.v

/bin/bash: line 1: yosys: command not found


# Vivado


So people mostly use TCL and no gui mode?
https://www.xilinx.com/support/documents/sw_manuals/xilinx2022_1/ug835-vivado-tcl-commands.pdf

```
Vivado% help
Vivado% help get_*
```



In [ ]:
%%bash
~/Xilinx/Vivado/2024.1/bin/vivado -mode tcl # interactive
vivado -mode batch -source <your_Tcl_script>

In [1]:
%%file /tmp/counter.v
module counter(
    input clk,
    input reset,
    output reg [3:0] count
);

always @(posedge clk or posedge reset) begin
    if (reset)
        count <= 4'b0000;
    else
        count <= count + 1;
end

endmodule


Overwriting /tmp/counter.v


In [8]:
%%file /tmp/counter_tb.v
`timescale 1ns / 1ps

module counter_tb;
    reg clk;
    reg reset;
    wire [3:0] count;

    // Instantiate the counter module
    counter uut (
        .clk(clk),
        .reset(reset),
        .count(count)
    );

    // Generate a clock signal
    initial begin
        clk = 0;
        forever #5 clk = ~clk; // 10ns period clock
    end

    // Stimulate the design
    initial begin
        reset = 1;
        #10; // Hold reset for 10ns
        reset = 0;
        #100; // Run simulation for 100ns
        $finish;
    end

    // Monitor the outputs
    initial begin
        $monitor("At time %t, count = %b", $time, count);
    end

endmodule


Writing /tmp/counter_tb.v


In [6]:
%%file /tmp/build.tcl
# non_project_build.tcl

# Set the part number for the target FPGA
set part_name "xc7a100tcsg324-1"
cd /tmp/

# Read the Verilog file
read_verilog "/tmp/counter.v"

# Set the target device
synth_design -top counter -part $part_name

# Run implementation
opt_design
place_design
route_design

# Write the bitstream
#write_bitstream -force "counter.bit"


Overwriting /tmp/build.tcl


In [7]:
!~/Xilinx/Vivado/2024.1/bin/vivado -mode batch -nojournal -source /tmp/build.tcl


****** Vivado v2024.1.1 (64-bit)
  **** SW Build 5094488 on Fri Jun 14 08:57:50 MDT 2024
  **** IP Build 5091682 on Fri Jun 14 16:55:04 MDT 2024
  **** SharedData Build 5094118 on Fri Jun 14 01:09:43 MDT 2024
  **** Start of session at: Fri Aug 30 10:09:44 2024
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /tmp/build.tcl
# set part_name "xc7a100tcsg324-1"
# cd /tmp/
# read_verilog "/tmp/counter.v"
# synth_design -top counter -part $part_name
Command: synth_design -top counter -part xc7a100tcsg324-1
Starting synth_design
Attempting to get a license for feature 'Synthesis' and/or device 'xc7a100t'
INFO: [Common 17-349] Got license for feature 'Synthesis' and/or device 'xc7a100t'
INFO: [Synth 8-7079] Multithreading enabled for synth_design using a maximum of 4 processes.
INFO: [Synth 8-7078] Launching helper process for spawning children vivado processes
INFO: [Synth 8-7075] Helper process

## arty

https://digilent.com/reference/programmable-logic/arty-a7/start

https://www.youtube.com/watch?v=Grs0gjeMPOY&ab_channel=AMDXilinx

In [79]:
! mkdir /tmp/blinky

mkdir: cannot create directory ‘/tmp/blinky’: File exists


In [80]:
%%file /tmp/blinky/blinky.v
module blinky(
    input wire clk,
    output reg led
);
    reg [24:0] counter;

    always @(posedge clk) begin
        counter <= counter + 1;
        led <= counter[24];
    end
endmodule

Overwriting /tmp/blinky/blinky.v


In [88]:
%%file /tmp/blinky/arty_a7.xdc
# Set the clock input and define I/O standard and pin location
set_property -dict {PACKAGE_PIN E3 IOSTANDARD LVCMOS33} [get_ports clk]
create_clock -period 10.00 [get_ports clk]

# Set the LED output and define I/O standard and pin location
set_property -dict {PACKAGE_PIN G6 IOSTANDARD LVCMOS33} [get_ports led]

# Set configuration voltage properties to avoid CFGBVS warning
set_property CFGBVS VCCO [current_design]
set_property CONFIG_VOLTAGE 3.3 [current_design]

Overwriting /tmp/blinky/arty_a7.xdc


Note that these cdx files are also tcl and a speiclzied version of sdc, synosys desifgn xconstraints
set_property -dict
https://github.com/Digilent/digilent-xdc/blob/master/Arty-A7-100-Master.xdc master file for arty board here
get_ports refers to names coming into top level verilog module.



In [86]:
%%file /tmp/run_vivado.tcl
# Set up the project
create_project arty_a7_blinky . -force -part xc7a35tcpg236-1
add_files blinky.v
add_files arty_a7.xdc

# Set top module and run synthesis
set_property top blinky [current_fileset]
synth_design -top blinky

# Implement the design
opt_design
place_design
route_design

# Generate the bitstream
write_bitstream -force arty_a7_blinky.bit

Overwriting /tmp/run_vivado.tcl


In [89]:
! cd /tmp/blinky && ~/Xilinx/Vivado/2024.1/bin/vivado -mode batch -source /tmp/run_vivado.tcl


****** Vivado v2024.1.2 (64-bit)
  **** SW Build 5164865 on Thu Sep  5 14:36:28 MDT 2024
  **** IP Build 5164407 on Fri Sep  6 08:18:11 MDT 2024
  **** SharedData Build 5164864 on Thu Sep 05 13:09:09 MDT 2024
  **** Start of session at: Thu Oct 31 16:21:21 2024
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /tmp/run_vivado.tcl
# create_project arty_a7_blinky . -force -part xc7a35tcpg236-1
# add_files blinky.v
# add_files arty_a7.xdc
# set_property top blinky [current_fileset]
# synth_design -top blinky
Command: synth_design -top blinky
Starting synth_design
Using part: xc7a35tcpg236-1
Attempting to get a license for feature 'Synthesis' and/or device 'xc7a35t'
INFO: [Common 17-349] Got license for feature 'Synthesis' and/or device 'xc7a35t'
INFO: [Device 21-403] Loading part xc7a35tcpg236-1
INFO: [Device 21-9227] Part: xc7a35tcpg236-1 does not have CEAM library.
INFO: [Synth 8-7079] Multi

In [ ]:
! cd /tmp/blinky && ~/Xilinx/Vivado/2024.1/bin/vivado 


leaf cells



# Soc
https://www.reddit.com/r/FPGA/comments/16138cd/litex_vs_fusesoc/
fusesoc litex
https://github.com/lowRISC/ibex-demo-system
https://github.com/lowRISC/ibex-demo-system-labs/blob/main/lab1.md
`sudo docker build . -t ibex -f container/Dockerfile` build docker image
```
sudo docker run -it --rm \
  -p 6080:6080 \
  -p 3333:3333 \
  -v $(pwd):/home/dev/demo:Z \
  ibex
  /bin/bash

./build/lowrisc_ibex_demo_system_0/sim-verilator/Vtop_verilator \
  --meminit=ram,./sw/c/build/demo/hello_world/demo

# ctrl-A \ to exit screen
```

https://github.com/lowRISC/opentitan/tree/master/hw/dv/verilator
https://github.com/lowRISC/opentitan/blob/master/hw/dv/dpi/uartdpi/uartdpi.c this is the uart code

http://localhost:6080/vnc.html

https://people.eecs.berkeley.edu/~alanmi/publications/2012/iwls12_lms.pdf lazy mans logic syntehsis





## LiteX

https://github.com/enjoy-digital/litex/wiki/Installation
https://github.com/litex-hub/fpga_101?tab=readme-ov-file

systemc-dev
picolibc

`litex_sim --cpu-type=vexriscv`

https://github.com/enjoy-digital/litex/tree/master/litex/soc/software/bios
https://github.com/enjoy-digital/litex/tree/master/litex/soc/cores/cpu/ibex
https://github.com/litex-hub/pythondata-cpu-ibex
https://github.com/litex-hub

hmm
  --rom-init ROM_INIT
          ROM init file (.bin or .json). (default: None)
  --ram-init RAM_INIT
          RAM init file (.bin or .json). (default: None)
sound kind of useful


https://www.controlpaths.com/2022/01/17/building-soc-litex/
`./digilent_arty.py --build --load`

https://github.com/picolibc/picolibc




## Amaranth


## FuseSoc
https://github.com/fusesoc/fusesoc-cores?tab=readme-ov-file
It's a yaml file thing that says how to run iverilog, synth, and what files are of interest, what is top level.
Seems like a lot of machinery...





# Analog
FPGA analog fronteneds.
ADC DAC board.



